In [2]:
# ============================================
# 01 - DATA CLEANING
# ============================================

import pandas as pd
from pathlib import Path
import numpy as np

# -------------------------
# Carregar dataset limpo da etapa 00
# -------------------------

DATA_PATH = Path('../data/internet_adoption_clean.csv')
df = pd.read_csv(DATA_PATH)

print("Formato inicial:", df.shape)
display(df.head())

# -------------------------
# Ajustar a coluna de data
# -------------------------

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Criar colunas adicionais de data
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# -------------------------
# Corrigir formato da coluna 5G_Rollout_Status
# -------------------------

# Substituir valores ausentes por "Unknown"
df['5G_Rollout_Status'] = df['5G_Rollout_Status'].fillna("Unknown")

# Normalizar texto
df['5G_Rollout_Status'] = (
    df['5G_Rollout_Status']
    .str.strip()
    .str.lower()
    .str.replace("-", " ")
    .replace("", "unknown")
)

# Exemplo de padronizações
df['5G_Rollout_Status'] = df['5G_Rollout_Status'].replace({
    "not started": "not_started",
    "in progress": "in_progress",
    "limited": "limited",
    "available": "available",
    "unknown": "unknown"
})

print("\nValores únicos do status 5G:")
print(df['5G_Rollout_Status'].unique())

# -------------------------
# Detectar e tratar outliers (Z-score)
# -------------------------

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Calcular Z-score
z_scores = (df[numeric_cols] - df[numeric_cols].mean()) / df[numeric_cols].std()

# Identificar outliers mais extremos (ex.: |z| > 4)
outliers = (np.abs(z_scores) > 4).any(axis=1)

print("\nQuantidade de outliers detectados:", outliers.sum())

# Remover outliers (opção típica em projetos)
df_cleaned = df[~outliers].copy()

print("Novo formato (sem outliers):", df_cleaned.shape)

# -------------------------
# Salvar dataset limpo
# -------------------------

OUTPUT_PATH = Path('../data/internet_adoption_clean_final.csv')
df_cleaned.to_csv(OUTPUT_PATH, index=False)

print("\nArquivo final salvo em:", OUTPUT_PATH)


Formato inicial: (6350, 28)


,Country,Date,Population,Internet_Penetration (%),Broadband_Speed (Mbps),Mobile_Broadband_Speed (Mbps),GDP_Per_Capita (USD),Education_Attainment_Index (%),Mobile_Data_Usage (GB per User),Digital_Investment (M USD),...,Device_Penetration (%),Cybersecurity_Incidents (Count),E_Commerce_Penetration (%),Government_Digital_Policy_Index (%),Network_Latency (ms),Cloud_Service_Adoption (%),IoT_Device_Density (per 1000 people),AI_Adoption_Index (%),Data_Privacy_Regulation_Strength (%),Energy_Consumption_for_Connectivity (kWh)
0,Algeria,2015-01-01,25258314,37.436774,46.839032,8.533871,5327.303548,45.489677,2.167742,164.128065,...,68.104839,4174,33.408387,37.136774,77.050645,27.834194,22.950645,15.652581,27.985161,248.871935
1,Algeria,2015-02-01,25258314,37.484286,47.227500,8.687500,5331.729286,45.555714,2.185357,165.386071,...,68.122500,3753,33.520000,37.128214,77.045000,27.856429,22.936429,15.681429,27.989643,248.757500
2,Algeria,2015-03-01,25258314,37.629677,47.436129,8.852903,5342.169032,45.608710,2.182581,166.200645,...,68.108065,4148,33.545806,37.155806,77.120968,27.870968,23.034839,15.686452,27.993548,248.524839
3,Algeria,2015-04-01,25258314,37.257667,47.500000,9.058333,5354.909667,45.643333,2.199000,166.618667,...,68.147000,4032,33.579333,37.120000,77.255333,27.880333,23.083667,15.706667,28.010000,248.497333
4,Algeria,2015-05-01,25258314,37.492258,47.659677,8.886452,5360.973226,45.662258,2.205484,167.409032,...,68.143548,4195,33.598710,37.155484,77.075161,27.930323,23.111290,15.729355,28.018065,248.983548



Valores únicos do status 5G:
['0' 'partial' 'planned' 'full']

Quantidade de outliers detectados: 6
Novo formato (sem outliers): (6344, 30)

Arquivo final salvo em: ../data/internet_adoption_clean_final.csv
